## Task 5 Extracting location data (Hotel, Restaurant, and Tourist sites)

### <span style="color:Red;"> I recommend that you should start in Task 1 to install the libraries needed and to avoid error running this notebook. Thank you

In [ ]:
pip install requests

In [ ]:
# Run this code to restart the kernel and apply the newly installed libraries.
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Importing libraries

In [170]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
from selenium.webdriver.common.keys import Keys
import platform
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.common.by import By
from selenium import webdriver
import time
import random
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.edge.service import Service

### To check OS if you are in Windows or Mac
##### <span style="color:blue;"> Please don't forget to run this code block and dont worry about this as it will not install anything on your laptop/PC as I already included in the submission folder the necessary driver

In [161]:
def chks_plat():
    if platform.system() == 'Windows':
        # Set the WebDriver executable path for Windows
        service  ='msedgedriver.exe'
    else:
        # Set the WebDriver executable path for macOS
        service ='msedgedriver'
        
    return service

In [162]:
# run this code to assign driver
CHKS = chks_plat()
# as Im running this code on windows Ive got the webdriver with .exe file extension
CHKS

'msedgedriver.exe'

## Population data from PSA

#### <span style="color:red;">The code below will always print File already exist</strong></span><span style="color:green;"> because the file is already been downloaded</span>

In [2]:
url = 'https://psa.gov.ph/system/files/phcd/Region%20VII.xlsx' # source
file_name = 'RegionVII.xlsx'

# Check if the file already exists, if not, it will pull the file from the PSA webpage
if not os.path.exists(file_name):
    with open(file_name, 'wb') as f:
        f.write(requests.get(url).content)
    print(f"File '{file_name}' downloaded successfully.")
else:
    print(f"File '{file_name}' already exists. Skipping download.")

File 'RegionVII.xlsx' already exists. Skipping download.


## Extracting population data
### <span style="color:blue;">Were gonna extract the population data from PSA.</span>

In [47]:
# The file path to PSA data
xlsx_file_path = 'RegionVII.xlsx'

In [48]:
# This function will rename columns and search for location of the population data
def rename_col(df, search_loc):
    
    df.rename(columns={'Unnamed: 0': 'City/Mun', 
                      'Unnamed: 1': 'Total Population', 
                      'Unnamed: 2': 'Household Population',
                      'Unnamed: 3': 'Numnber of Household'}, inplace=True)
    
    for index, row in df.iterrows():
        if row['City/Mun'] == search_loc:
            return row 

### <span style="color:blue;">Cebu City, Lapu lapu City, Moalboal Mun, Mandaue City</span>
###### were gonna store the population value in separate variable, one for each group

In [81]:
# In this block we are going to change the header of the dataframe and search for the specific row on where the
# total population data on each Muncipality/City will be extracted.

df_cebcty = pd.read_excel(xlsx_file_path, sheet_name='City of Cebu', skiprows = 6)
cebcty = rename_col(df_cebcty, 'CITY OF CEBU (Capital)')

df_lapcty = pd.read_excel(xlsx_file_path, sheet_name='City of Lapu-Lapu', skiprows = 6)
lapcty = rename_col(df_lapcty, 'CITY OF LAPU-LAPU (OPON)')

df_moalmun = pd.read_excel(xlsx_file_path, sheet_name='Cebu', skiprows=6)
moalmun = rename_col(df_moalmun, 'MOALBOAL')

df_mancty = pd.read_excel(xlsx_file_path, sheet_name='City of Mandaue', skiprows=6)
mancty = rename_col(df_mancty, 'CITY OF MANDAUE')

### <span style="color:blue;">Province</span>
#### <span style="color:blue;">This group is consisting of 7 count of Municipalities ('Badian', 'Dalaguete', 'Oslob' (2 counts), 'Samboan', 'Sibonga', and 'Santa Fe') which doesnt belong to the 4 groups above but they are all located in the same Province which is Cebu</span>


###### The location data I had scraped from the website in Task 1,2, and 3 some of them was inaccurate, so  I have to update  the "Lindogon Simala" to "(SIBONGA)" and "Sumilon Island" to "(OSLOB).

In [313]:
df_prov = pd.read_excel(xlsx_file_path, sheet_name='Cebu', skiprows=6)
mun_list = ['BADIAN', 'DALAGUETE', 'OSLOB', 'SAMBOAN', 'SIBONGA', 'SANTA FE']
list_prov = []
list_prov_val = []

# In this loop we are going to change the header of the dataframe and search for the specific row on where the
# total population data on each Muncipality/City will be extracted.
for loc in mun_list:
    list_prov.append(rename_col(df_prov, loc))

# to get every total population value and store it on a list for reg7
for val in list_prov:
    list_reg_val.append(val['Total Population'])

### <span style="color:blue;"> Summary dataframe for Population data grouped by City/Mun/Province on task 4 groupings</span>
#### The Municipalities with "Group name" containing "(Prov)" are part of a single group in task 4 and in here, I intend to display them by Municipality, showing their respective population numbers without aggregating them.

In [314]:
pop_data = {'Total Population': [cebcty.iloc[1], lapcty.iloc[1], moalmun.iloc[1], mancty.iloc[1], 
                                 list_reg_val[0], list_reg_val[1], list_reg_val[2], list_reg_val[3], 
                                 list_reg_val[4], list_reg_val[5]]}

index = ['Cebu City', 'Lapu lapu City', 'Moalboal Mun', 'Mandaue City', 
         'Badian Mun (Prov)', 'Dalaguete Mun (Prov)', 'Oslob Mun (Prov)', 
         'Samboan Mun (Prov)', 'Sibonga Mun (Prov)', 'Santa Fe. Mun (Prov)']

# creating data frame for summary data
df_pop_data = pd.DataFrame(pop_data, index=index)
df_pop_data.index.name = 'Group'
df_pop_data

,Total Population
Group,
Cebu City,964169.0
Lapu lapu City,497604.0
Moalboal Mun,36930.0
Mandaue City,364116.0
Badian Mun (Prov),43735.0
Dalaguete Mun (Prov),74596.0
Oslob Mun (Prov),29264.0
Samboan Mun (Prov),20373.0
Sibonga Mun (Prov),53424.0


## Extracting primary and secondary language data from data.humdata.org

#### Primary Language source

In [315]:
url = 'https://data.humdata.org/dataset/4383caa9-b4e0-4608-80f6-2b74482fe8bd/resource/fb35553c-e3bc-4044-9729-8351ad4650f5/download/ph_lang_admin2_v01.csv' # source
file_name = 'ph_lang_admin2v01.csv'

# Check if the file already exists, if not, it will pull the file from the data.humdata.org webpage
if not os.path.exists(file_name):
    with open(file_name, 'wb') as f:
        f.write(requests.get(url).content)
    print(f"File '{file_name}' downloaded successfully.")
else:
    print(f"File '{file_name}' already exists. Skipping download.")

File 'ph_lang_admin2v01.csv' already exists. Skipping download.


#### Secondary Language source

In [316]:
url = 'https://data.humdata.org/dataset/4383caa9-b4e0-4608-80f6-2b74482fe8bd/resource/851e0755-14ff-43f4-ab14-04bcab7bf9a3/download/ph_lang_admin0_v01.csv' # source
file_name = 'ph_lang_admin01v01.csv'

# Check if the file already exists, if not, it will pull the file from the data.humdata.org webpage
if not os.path.exists(file_name):
    with open(file_name, 'wb') as f:
        f.write(requests.get(url).content)
    print(f"File '{file_name}' downloaded successfully.")
else:
    print(f"File '{file_name}' already exists. Skipping download.")

File 'ph_lang_admin01v01.csv' already exists. Skipping download.


In [317]:
# path to the pulled file from data.humdata.org - Primary language
csv_file_path1 = 'ph_lang_admin2v01.csv'

# path to the pulled file from data.humdata.org - Secondary language
csv_file_path2 = 'ph_lang_admin01v01.csv'

#### <span style="color:blue;">In here were gonna see that for Cebu, the primary or the main language is 'Cebuano'. Cebu is also a part of Region 7 together with Bohol, Negros Oriental, and Siquijor.</span>

In [318]:
ceb_data1= []

data_lang1 = pd.read_csv(csv_file_path1, encoding='latin1')

# will search for the Cebu in the data frame
for index, row in data_lang1.iterrows():
    if row['admin2_pcode'] == 'Cebu':
        ceb_data1.append(row)
        
df_ceb_data1 = pd.DataFrame(ceb_data1)
df_ceb_data1

,admin2_name,admin2_pcode,admin1_name,admin1_pcode,admin0_name,admin0_pcode,number_of_named_languages,main_language,main_language_share,Adasen,...,Other languages,pop_total,pop_male,pop_female,literacy_all,literacy_male,literacy_female,language_data_yn,data_confidence,notes
34,PH072200000,Cebu,Central Visayas (Region VII),PH070000000,Philippines (the),PH,5,Cebuano,0.876,0,...,0.001,4632359,2337313,2295046,0.983,0.983,0.983,Y,High,NaN


#### <span style="color:blue;">Secondary language</span>
##### Tagalog is the most dominant language based on the data from data.humdata.org, so I will use this as the secondary language

In [319]:
ceb_data2= []

data_lang2 = pd.read_csv(csv_file_path2, encoding='latin1')

# will search for the Phillipines in the data frame
for index, row in data_lang2.iterrows():
    if row['ï»¿admin0_name'] == 'Philippines (the)':
        ceb_data2.append(row)
        
df_ceb_data2 = pd.DataFrame(ceb_data2)
df_ceb_data2

,ï»¿admin0_name,admin0_pcode,number_of_named_languages,main_language,main_language_share,Akeanon,Bikol,Bisaya/Binisaya,B'laan/Blaan language group,CapizeÃ±o,...,Other languages,pop_total,pop_male,pop_female,literacy_all,literacy_male,literacy_female,language_data_yn,data_confidence,notes
1,Philippines (the),PH,41,Tagalog,0.533,0.004,0.032,0.118,0.002,0.005,...,0.006,100979303,51069962,49909341,0.971,0.97,0.972,Y,High,NaN


### <span style="color:blue;">Updated Summary dataframe</span>

In [320]:
if 'Primary Language' not in df_pop_data.columns or 'Secondary Language' not in df_pop_data.columns:
    df_pop_data.insert(loc=1, column='Primary Language', value='Cebuano')
    df_pop_data.insert(loc=2, column='Secondary Language', value='Tagalog') 

In [321]:
df_pop_data

,Total Population,Primary Language,Secondary Language
Group,,,
Cebu City,964169.0,Cebuano,Tagalog
Lapu lapu City,497604.0,Cebuano,Tagalog
Moalboal Mun,36930.0,Cebuano,Tagalog
Mandaue City,364116.0,Cebuano,Tagalog
Badian Mun (Prov),43735.0,Cebuano,Tagalog
Dalaguete Mun (Prov),74596.0,Cebuano,Tagalog
Oslob Mun (Prov),29264.0,Cebuano,Tagalog
Samboan Mun (Prov),20373.0,Cebuano,Tagalog
Sibonga Mun (Prov),53424.0,Cebuano,Tagalog


## Extracting reviews/description of each City/Municipality

##### <span style="color:blue;">Below is a list of URL's to a specific page in TripAdvisor on where we will scrappe data

In [322]:
desrev = ['https://www.tripadvisor.com/Tourism-g298460-Cebu_City_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g298461-Lapu_Lapu_Mactan_Island_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g663539-Moalboal_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g608518-Mandaue_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g2140591-Badian_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g1498929-Dalaguete_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g3226128-Oslob_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g7351247-Samboan_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g3814813-Sibonga_Cebu_Island_Visayas-Vacations.html',
         'https://www.tripadvisor.com/Tourism-g2450335-Santa_Fe_Bantayan_Island_Cebu_Island_Visayas-Vacations.html']

##### <span style="color:blue;">Below is a function to scrappe review/description data

In [323]:
# Scrapping Tourist sites reviews
def scrape_rev(url_x):    
    service = Service(executable_path=CHKS) 
    options = webdriver.EdgeOptions()
    options.add_argument("--headless")
    driver = webdriver.Edge(service=service, options=options)

    # Use Selenium to open the webpage
    driver.get(url_x)

    # Number of times to scroll down
    num_scrolls = random.randint(5,6)

    # Delay between each scroll action (in seconds)
    scroll_delay = random.randint(2,3)  # Adjust as needed

    # Perform scrolling
    for i in range(num_scrolls):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(scroll_delay)

    # Retrieve the entire page source
    page_source = driver.page_source
    
    rev_elem_list = []

    # Parse the page source with BeautifulSoup and Find the element                                                
    soup = BeautifulSoup(page_source, 'html.parser')
    name_element = soup.find_all(['span', 'div'], class_=['biGQs _P fiohW fOtGX','UpQlM'])
    for element in name_element:
        if 'from $' not in element.text.strip():
            rev_elem_list.append(element.text.strip())
        else:
            pass

    # Close the browser
    driver.quit()
            
    return rev_elem_list

##### <span style="color:blue;">The data output of this scrapping is not exactly a review but rather it is more of a description/guide on where is the best hotel, restaurant and tourist site in the place. It also gives customer advices on what is the best time of the year to travel in the place.

In [247]:
# Batch 1 scrapping review/description data
rev_data = []
link_ctr = 0

for link in desrev:
    link_ctr+=1
    if link_ctr <= 10:
        print(f'URL No. {link_ctr} out of 10, Please wait . . .')
        rev_data.append(scrape_rev(link))
        time.sleep(random.randint(4,6))
        if link_ctr == 10:
            print('Done')

URL No. 1 out of 10, Please wait . . .
URL No. 2 out of 10, Please wait . . .
URL No. 3 out of 10, Please wait . . .
URL No. 4 out of 10, Please wait . . .
URL No. 5 out of 10, Please wait . . .
URL No. 6 out of 10, Please wait . . .
URL No. 7 out of 10, Please wait . . .
URL No. 8 out of 10, Please wait . . .
URL No. 9 out of 10, Please wait . . .
URL No. 10 out of 10, Please wait . . .
Done


##### <span style="color:blue;">Result from scrapping

In [324]:
clnd_rev_data = []

# removing the unwanted/unnecessary data
for data in rev_data:
    if 'Terms of UsePrivacy and Cookies StatementCookie consentSite MapHow the site worksContact us' in data:
        clnd_rev_data.append(data[:-1])
        print(f'{data[:-1]}\n\n')
        
# below is the printed data from scraping

['We recommend staying at one of the most popular hotels in Cebu City, which include: Quest Hotel and Conference Center - CebuRadisson Blu CebuWaterfront Cebu City Hotel & CasinoFili Hotel CebuSeda Ayala Center Cebu', 'Some of the most popular restaurants in Cebu City include: FeriaThe Pig & PalmVikings Luxury Buffet, SM City CebuHouse of LechonAnzani', 'Cebu City is known for some of its popular attractions, which include: Temple of LeahAyala Center CebuSirao Flower GardenCebu Ocean ParkBasilica del Santo Niño', 'We recommend checking out these popular tours when looking for something to do in Cebu City: Whale Shark Encounter and Sumilon Sandbar with Kawasan FallsPrivate Cebu & Lapu-Lapu City Tour with Uphill Spots & Lunch at House of LechonBohol Day Tour with Round-Trip Transfers from CebuWhale Shark Snorkeling w/ Tumalog Falls & Canyoneering in KawasanPescador Island with Sardines & Turtle and Canyoneering in Kawasan Falls', "If you're a more budget-conscious traveler, then you may 

### <span style="color:blue;">Updated Summary by inserting review/description</span>

In [325]:
# inserting the review/description column and values
if 'Review/Description' not in df_pop_data.columns:
    df_pop_data.insert(loc=1, column='Review/Description', value=clnd_rev_data)
    

In [326]:
# there is no data available from the target website on Sibonga Municipality
df_pop_data

# I cant find also the data for available transportation in every city

,Total Population,Review/Description,Primary Language,Secondary Language
Group,,,,
Cebu City,964169.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Lapu lapu City,497604.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Moalboal Mun,36930.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Mandaue City,364116.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Badian Mun (Prov),43735.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Dalaguete Mun (Prov),74596.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Oslob Mun (Prov),29264.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Samboan Mun (Prov),20373.0,[We recommend staying at one of the most popul...,Cebuano,Tagalog
Sibonga Mun (Prov),53424.0,[],Cebuano,Tagalog


In [328]:
# saving data frame into csv file
df_pop_data.to_csv('Tourist_site_Task5.csv')